In [1]:
#! /usr/bin/env python3
import numpy as np
# install the ortools just by running pip3 install ortools
from ortools.linear_solver import pywraplp
from collections import defaultdict

In [2]:
class Graph(object):
    """ Graph data structure, undirected by default. """

    def __init__(self, connections, node_weight, directed=False):
        self._graph = defaultdict(set)
        self._directed = directed
        self.add_connections(connections)
        self.node_weight = dict()
        self.edge_price = dict()
        for idx,w in enumerate(node_weight):
            self.node_weight[idx+1] = w
    
    def increase_edge(self,edge):
        a = edge[0]
        b = edge[1]
        pa = self.node_weight[a]
        pb = self.node_weight[b]
        if pa<pb:
            inc_price = pa
        else:
            inc_price = pb
        if edge not in self.edge_price.keys():
            self.edge_price[edge] = 0
        self.edge_price[edge] += inc_price
        self.node_weight[a] -= inc_price
        self.node_weight[b] -= inc_price
        return inc_price
    
    def is_tight(self, node):
        '''
        p_sum = 0
        ad_edge = list()
        for n in g.__graph[node]:
            ad_edge.add({node,n})
        for edge in ad_edge:
            a = edge.pop()
            b = edge.pop()
            if b<a:
                tmp = b
                b = a
                a = tmp
            p_sum += price[(a,b)]
        '''
        if self.node_weight[node] == 0:
            return True
        else:
            return False

    def add_connections(self, connections):
        """ Add connections (list of tuple pairs) to graph """

        for node1, node2 in connections:
            self.add(node1, node2)

    def add(self, node1, node2):
        """ Add connection between node1 and node2 """

        self._graph[node1].add(node2)
        if not self._directed:
            self._graph[node2].add(node1)

    def remove(self, node):
        """ Remove all references to node """

        for n, cxns in self._graph.iteritems():
            try:
                cxns.remove(node)
            except KeyError:
                pass
        try:
            del self._graph[node]
        except KeyError:
            pass

    def is_connected(self, node1, node2):
        """ Is node1 directly connected to node2 """

        return node1 in self._graph and node2 in self._graph[node1]

    def find_path(self, node1, node2, path=[]):
        """ Find any path between node1 and node2 (may not be shortest) """

        path = path + [node1]
        if node1 == node2:
            return path
        if node1 not in self._graph:
            return None
        for node in self._graph[node1]:
            if node not in path:
                new_path = self.find_path(node, node2, path)
                if new_path:
                    return new_path
        return None

    def __str__(self):
        return '{}({})'.format(self.__class__.__name__, dict(self._graph))

In [3]:
def vertex_cover_approx(graph_mat, w):
    '''
    % *********************************************************************** %
    % Vertex Cover Approximation Algorithm (Pricing Method).
    % -------
    % INPUT :
    % -------
    %   graph_mat : list[tuple(2)*edge_num], Integer :: each row denotes an edge
    %       e.g., [1 2; 2 3; 3 4; 4 1; 2 4]
    %   w : list[vertex_num], Double :: node w
    %       e.g., [3 4 3 5]
    % -------
    % OUTPUT:
    % -------
    %   vertex_set_indices : matrix(1, ?), Integer :: vertex set indices generated sequentially
    %       e.g., [1 2 3]
    %   prices : matrix(1, ?), Double :: prices generated sequentially
    %       e.g., [3 1 2]
    % *********************************************************************** %
    '''
    g = Graph(list(graph_mat), w)
    uncovered = graph_mat
    price = list()
    tight_vertices = set()
    while len(uncovered)>0:
        to_covered = set()
        for edge in uncovered:
            if edge not in to_covered and (not g.is_tight(edge[0]) or  not g.is_tight(edge[1])):
                # print(edge)
                price.append(g.increase_edge(edge))
            if g.is_tight(edge[0]):
                tight_vertices.add(edge[0])
                for n2 in g._graph[edge[0]]:
                    if (n2,edge[0]) in uncovered:
                        to_covered.add((n2,edge[0]))
                    if (edge[0],n2) in uncovered:
                        to_covered.add((edge[0],n2))
            if g.is_tight(edge[1]):
                tight_vertices.add(edge[1])
                for n2 in g._graph[edge[1]]:
                    if (n2,edge[1]) in uncovered:
                        to_covered.add((n2,edge[1]))
                    if (edge[1],n2) in uncovered:
                        to_covered.add((edge[1],n2))    
        uncovered = uncovered.difference(to_covered)
        total = 0
        for v in tight_vertices:
            total+=w[v-1]
    return tight_vertices,total,price

In [4]:
def def_var(vector_len, solver, bounds):
    var = list()
    for idx,bound in zip(range(vector_len),bounds):
        var.append(solver.IntVar(bound[0],bound[1], 'x%d' % idx))
    return var

def constraint_process(A, b, solver, var):
    '''
    Ax<=b
    input: A: matrix mxn
    b: vector n
    '''
    constraint = list()
    for row,value in zip(A,b):
        # print(row,value)
        constraint_tmp = solver.Constraint(-solver.infinity(), value)
        constraint.append(constraint_tmp)
        for v,var_ in zip(row,var):
            # print(v,var_,type(var_))
            constraint_tmp.SetCoefficient(var_, v)
    return constraint
    # np.apply_along_axis(lambda x:print(x), axis=1, arr=A)
    
def set_obj(w,solver,var):
    '''
    minimize wTx
    '''
    objective = solver.Objective()
    for var_, weight in zip(var, w):
        # print(var_,weight)
        objective.SetCoefficient(var_, float(weight))
    return objective

In [5]:
def vertex_cover_LP(graph_mat, w):
    '''
    % *********************************************************************** %
    % Vertex Cover Approximation Algorithm (Pricing Method).
    % -------
    % INPUT :
    % -------
    %   graph_mat : list[tuple(2)*edge_num], Integer :: each row denotes an edge
    %       e.g., [1 2; 2 3; 3 4; 4 1; 2 4]
    %   w : list[vertex_num], Double :: node w
    %       e.g., [3 4 3 5]
    % -------
    % OUTPUT:
    % -------
    %   vertex_set_indices : matrix(1, ?), Integer :: vertex set indices generated sequentially
    %       e.g., [1 2 3]
    % *********************************************************************** %
    '''
    import numpy as np
    from scipy.optimize import linprog
    w = np.array(w)
    num_vertices = len(w)
    num_edges = len(graph_mat)
    A = np.zeros((num_edges,num_vertices))
    for idx, edge in enumerate(graph_mat):
        A[idx,edge[0]-1] = 1
        A[idx,edge[1]-1] = 1
    x0_bounds = np.zeros(num_vertices)
    x1_bounds = np.ones(num_vertices)
    b = np.ones(num_edges)
    solver = pywraplp.Solver('SolveIntegerProblem',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    var = def_var(len(w),solver,list(zip(x0_bounds, x1_bounds)))
    constraint_process(-A, -b, solver, var)
    set_obj(w,solver,var)
    # print(len(x1_bounds), len(w))
    # print(w,-A.transpose())
    # res = linprog(w, A_ub=-A, b_ub=-ones, bounds=list(zip(x0_bounds, x1_bounds)),options={"disp": True})
    result_status = solver.Solve()
    print(result_status== pywraplp.Solver.OPTIMAL)
    result = set()
    total = 0
    for idx,variable in enumerate(var):
        # print(variable.solution_value())
        if variable.solution_value() == 1:
            result.add(idx+1)
            total += w[idx]
    return result,total

In [6]:
graph_mat = set([(1,2),(2,3),(3,4),(4,1),(2,4)])
w = [3,4,3,5]
print("LP-based (set total_weight): ", vertex_cover_LP(graph_mat, w))
print("approx: (set total_weight)", vertex_cover_approx(graph_mat,w)[0:2])

True
LP-based (set total_weight):  ({2, 4}, 9)
approx: (set total_weight) ({1, 2, 3}, 10)


In [7]:
graph_mat = set([(1,2),(1,3),(1,6),(2,4),(2,5),(3,4),(3,5),(4,6),(5,6)])
w = list(range(11,0,-2))
print("LP-based (set total_weight): ", vertex_cover_LP(graph_mat, w))
print("approx: (set total_weight)", vertex_cover_approx(graph_mat,w)[0:2])

True
LP-based (set total_weight):  ({2, 3, 6}, 17)
approx: (set total_weight) ({1, 2, 3, 4, 6}, 33)


In [8]:
graph_mat =set()
for i in range(2,1002):
    graph_mat.add((1,i))
weights = [500,] + [1,]*1000
print("LP-based (set total_weight): ", vertex_cover_LP(graph_mat, weights))
print("approx: (set total_weight)", vertex_cover_approx(graph_mat,weights)[0:2])

True
LP-based (set total_weight):  ({1}, 500)
approx: (set total_weight) ({1, 3, 4, 6, 8, 10, 13, 15, 16, 18, 21, 22, 25, 28, 29, 31, 33, 37, 39, 40, 41, 43, 44, 46, 49, 51, 52, 54, 55, 56, 58, 62, 64, 65, 67, 68, 70, 72, 74, 77, 80, 82, 86, 89, 90, 93, 95, 98, 101, 103, 105, 107, 108, 110, 113, 115, 116, 119, 120, 122, 123, 124, 126, 128, 130, 133, 134, 137, 141, 143, 146, 149, 151, 153, 155, 156, 158, 159, 161, 163, 164, 167, 168, 170, 171, 174, 176, 177, 179, 180, 182, 184, 186, 189, 191, 192, 194, 195, 198, 201, 202, 205, 207, 208, 210, 213, 215, 217, 219, 220, 223, 225, 227, 228, 229, 231, 232, 235, 236, 238, 241, 243, 244, 246, 248, 250, 253, 256, 258, 261, 263, 265, 267, 268, 270, 271, 273, 276, 277, 279, 280, 283, 284, 286, 288, 289, 291, 292, 294, 296, 298, 301, 304, 306, 310, 313, 314, 317, 319, 320, 322, 325, 329, 331, 332, 335, 337, 338, 341, 343, 344, 345, 347, 348, 350, 353, 355, 356, 358, 360, 362, 365, 366, 368, 370, 371, 372, 374, 378, 381, 383, 385, 389, 391, 392, 393